# Strategy Optimization

This notebook demonstrates parameter optimization and walk-forward analysis.

## What You'll Learn
1. Grid search optimization
2. Walk-forward analysis (out-of-sample testing)
3. Avoiding overfitting
4. Position sizing

In [ ]:
import sys
import os
from datetime import date

sys.path.insert(0, os.path.abspath('..'))

from stocksimulator.data import load_from_csv
from stocksimulator.core.backtester import Backtester
from stocksimulator.optimization import GridSearchOptimizer, WalkForwardAnalyzer
from stocksimulator.optimization.position_sizing import KellyCriterion, FixedFractional
from stocksimulator.strategies import MomentumStrategy

## Load Data

In [ ]:
spy_data = load_from_csv('sp500_stooq_daily.csv', 'SPY', '../historical_data')
print(f"✓ Loaded {len(spy_data.data)} data points")

end_date = spy_data.data[-1].date
start_date = date(end_date.year - 5, end_date.month, end_date.day)

## Grid Search Optimization

Find optimal parameters by testing all combinations.

In [ ]:
# Create optimizer
optimizer = GridSearchOptimizer(
    backtester=Backtester(initial_cash=100000.0),
    optimization_metric='sharpe_ratio'  # Optimize for Sharpe ratio
)

# Define parameter grid
param_grid = {
    'lookback_days': [60, 126, 252],  # 3, 6, 12 months
    'top_n': [1],  # Hold top 1 position
    'equal_weight': [True]
}

print("Running grid search optimization...")
print(f"Testing {3} parameter combinations\n")

results = optimizer.optimize(
    strategy_class=MomentumStrategy,
    param_grid=param_grid,
    market_data={'SPY': spy_data},
    start_date=start_date,
    end_date=end_date,
    top_n=3
)

## Analyze Best Parameters

In [ ]:
print("\nTOP 3 PARAMETER COMBINATIONS")
print("=" * 70)

for i, result in enumerate(results, 1):
    summary = result.backtest_result.get_performance_summary()
    print(f"\n{i}. Parameters: {result.parameters}")
    print(f"   Sharpe Ratio: {result.metric_value:.3f}")
    print(f"   Ann. Return:  {summary['annualized_return']:.2f}%")
    print(f"   Max Drawdown: {summary['max_drawdown']:.2f}%")

## Walk-Forward Analysis

Test strategy robustness with out-of-sample testing.

Process:
1. Train on period 1, test on period 2
2. Train on period 2, test on period 3
3. Repeat walking forward through time

In [ ]:
# Walk-forward analyzer
wf_analyzer = WalkForwardAnalyzer(
    backtester=Backtester(initial_cash=100000.0)
)

print("Running walk-forward analysis...")
print("This may take a few minutes...\n")

wf_result = wf_analyzer.analyze(
    strategy_class=MomentumStrategy,
    param_grid={'lookback_days': [60, 126, 252]},
    market_data={'SPY': spy_data},
    train_days=504,  # 2 years training
    test_days=126,   # 6 months testing
    step_days=63     # 3 months step
)

## Walk-Forward Results

In [ ]:
wf_summary = wf_result.get_summary()

if wf_summary:
    print("\nWALK-FORWARD ANALYSIS RESULTS")
    print("=" * 70)
    print(f"Number of periods tested: {wf_summary['num_periods']}")
    print(f"\nOut-of-sample performance:")
    print(f"  Average return:      {wf_summary['avg_return']:.2f}%")
    print(f"  Average Sharpe:      {wf_summary['avg_sharpe']:.3f}")
    print(f"  Average max DD:      {wf_summary['avg_max_drawdown']:.2f}%")
    print("\nThis shows how the strategy performs on unseen data!")
else:
    print("Not enough data for walk-forward analysis")

## Position Sizing

Determine optimal position sizes using Kelly Criterion and Fixed Fractional.

In [ ]:
# Kelly Criterion
kelly = KellyCriterion(fraction=0.5)  # Half-Kelly for safety

# Example: Strategy with 60% win rate, 5% avg win, 3% avg loss
kelly_position = kelly.calculate_position_size(
    account_value=100000,
    win_rate=0.60,
    avg_win=0.05,
    avg_loss=0.03
)

print("\nKELLY CRITERION POSITION SIZING")
print("=" * 70)
print(f"Account value:    ${100000:,.2f}")
print(f"Win rate:         60%")
print(f"Avg win:          5%")
print(f"Avg loss:         3%")
print(f"\nRecommended size: ${kelly_position:,.2f} ({kelly_position/100000*100:.1f}% of account)")
print("\n(Using half-Kelly for reduced risk)")

In [ ]:
# Fixed Fractional
fixed_frac = FixedFractional(risk_pct=0.02)  # Risk 2% per trade

ff_position = fixed_frac.calculate_position_size(
    account_value=100000,
    stop_loss_pct=0.05  # 5% stop loss
)

print("\nFIXED FRACTIONAL POSITION SIZING")
print("=" * 70)
print(f"Account value:    ${100000:,.2f}")
print(f"Risk per trade:   2%")
print(f"Stop loss:        5%")
print(f"\nRecommended size: ${ff_position:,.2f} ({ff_position/100000*100:.1f}% of account)")

## Key Takeaways

1. **Grid Search**: Finds best parameters but can overfit
2. **Walk-Forward**: Tests on unseen data to prevent overfitting
3. **Position Sizing**: Manages risk and maximizes growth

Always validate with out-of-sample testing!